In [58]:
from gwosc.datasets import event_gps
from gwpy.timeseries import TimeSeries, TimeSeriesDict
from gwpy.frequencyseries import FrequencySeries
from gwpy.signal import filter_design
from gwpy.plot import Plot
from scipy.special import erf
from ipywidgets import interactive_output, VBox, HBox, FloatSlider, Checkbox, FloatLogSlider, FloatRangeSlider, FloatText

import numpy as np
import matplotlib.pyplot as plt
import plotfancy as pf
pf.housestyle_rcparams()

True

In [89]:
def lrm(data, base_window=10, log_base=10):
    data = np.asarray(data)
    is_1d = False

    if data.ndim == 1:
        data = data[:, np.newaxis]
        is_1d = True

    n, m = data.shape
    mean = np.zeros_like(data)
    std = np.zeros_like(data)

    for i in range(n):
        window_size = max(1, int(np.floor(np.log(i + 1) / np.log(log_base)) * base_window))
        start = max(0, i - window_size + 1)
        window = data[start:i + 1]
        mean[i] = window.mean(axis=0)
        std[i] = window.std(axis=0, ddof=0)  # population std (ddof=0), change to ddof=1 if sample std

    if is_1d:
        return mean.ravel(), std.ravel()
    return np.array([mean, std])

In [2]:
gps = event_gps('GW150914')
data = TimeSeries.fetch_open_data('L1', gps-5, gps+5)

In [141]:
def gs(x, centre=0, width=1, softness=0.1):
    return 0.5 * (erf((x - (centre - width/2)) / softness) - erf((x - (centre + width/2)) / softness))

def gslr(x, left, right, softness=0.1):
    return 0.5 * (erf((x - (left)) / softness) - erf((x - (right)) / softness))


gridlims = [1,1000]
res = len(data.psd())

def process_and_plot(lr,s, notch_b, notch_fun=60):
    grid = np.linspace(gridlims[0], gridlims[1], res)
    dat = gslr(grid, lr[0], lr[1],softness=s)
    
    notch_f = notch_fun*np.arange(1,7)

    if notch_b:
        notch_mask = np.ones(len(grid))
        for notch in notch_f:
            notch_mask = notch_mask*(1-gs(grid, width=2, softness=0.1, centre=notch))
    
        dat = dat*notch_mask

    fig, ax1 = pf.create_plot((4,3))
    ax1.set_xlim(gridlims)
    ax1.set_ylim(-0.05,1.05)
    ax1.plot(grid,dat, color='black', lw=2, zorder=10)
    ax1.set_xscale('log')

    ax2 = ax1.twinx()
    ax2.plot(data.psd(), color='#77aca2', alpha=0.6, zorder=-10)
    ax2.set_yscale('log')
    ax2.set_ylabel('Noise PSD')

    ax1.set_ylabel('$\exp($Gain [dB]$)$')
    ax1.set_xlabel('Frequency [Hz]')
    pf.fix_plot([ax1,ax2])
    ax1.tick_params(top=True,right=False, direction='in', length=7, which='major')
    ax1.tick_params(top=True,right=False, direction='in', length=4, which='minor')

    ax3 = fig.add_axes((1.35,0,1,1))
    # global gw
    gw = data.psd()*dat
    gwrolling = lrm(gw)
    ax3.loglog(grid,gwrolling[0], color='#ff004f')
    # ax3.fill_between(grid,gwrolling[0]-gwrolling[1],gwrolling[0]+gwrolling[1], color='#ff004f', alpha=0.3)
    ax3.set_ylim(5e-48, 5e-43)
    ax3.set_xlabel('Frequency [Hz]')

    ax4 = fig.add_axes((0,-1.3,2,1))
    global td_gw
    td_gw = np.fft.irfft(gw)

    tgrid = np.arange(0,4,np.float64(data.dt))
    ax4.plot(np.array(data.times)-data.epoch.value,td_gw)
    # ax4.plot(np.array(data.times)-data.epoch.value,np.array(data.data))
    ax4.set_xlim([4.95,5.05])

    pf.fix_plot([ax3,ax4])
    plt.show()
    plt.close(fig)

#### - OLD - ####
# l_slider = FloatLogSlider(min=0, max=3, step=0.01, value=np.log10(25), description='BP Left')
# r_slider = FloatLogSlider(min=0, max=3, step=0.01, value=np.log10(300), description='BP Right')

#### - SLIDERS - ####
lr_slider = FloatRangeSlider(min=1, max=1000, step=0.01, value=[24,300], description='BP')
s_slider = FloatLogSlider(min=-1, max=2, step=0.1, value=-.5, description='Softness')
n_slider = FloatLogSlider(min=0, max=3, step=0.001, value=60, description='Ground H0')
n_checkbox = Checkbox(value=False, description='Notches On?')

#### - PLOTTING - ####
out = interactive_output(process_and_plot, {'lr': lr_slider, 's':s_slider, 'notch_b':n_checkbox, 'notch_fun':n_slider})
VBox([lr_slider, s_slider,n_checkbox,n_slider, out])



<>:35: SyntaxWarning: invalid escape sequence '\e'
<>:35: SyntaxWarning: invalid escape sequence '\e'
/var/folders/6b/x84gj0ds47n3hjglw1wq1j8m0000gn/T/ipykernel_94362/3227418917.py:35: SyntaxWarning: invalid escape sequence '\e'
  ax1.set_ylabel('$\exp($Gain [dB]$)$')
